In [ ]:
! pip install networkx --quiet
! pip install GitPython --quiet
! pip install pandas --quiet
! pip install pyvis --quiet

In [68]:
import sys
sys.path.append('../src')

In [69]:
from weeks import Weeks
from ravendb_git import RavenDB_git

In [70]:
start, _ = Weeks.get_previous(12)
_, end = Weeks.get_current()

ravendb = RavenDB_git("~/Documents/GitHub/ravendb")
commits = ravendb.get_commits("v6.0", start, end)

In [71]:
f_commits = commits.filter_by(by='root', value='src')

In [72]:
f_commits.get_summary_by_author()

,author,changes,perc
1,Damian Olszewski,34071,44.326919
0,Arkadiusz Paliński,22243,28.938501
2,Federico Lois,5880,7.649975
22,shiranshalom,2006,2.609838
8,Maciej Aszyk,1834,2.386064
12,Michał Leśniak,1696,2.206523
6,Karmel Indych,1384,1.800606
11,Mateusz Bartosik,1175,1.528694
13,Paweł Pekról,1125,1.463643
18,efrat@ravendb.net,1050,1.366067


In [73]:
projects = f_commits.get_summary_by_code_project()

In [74]:
projects

,code_project,changes,perc
4,Raven.Studio,45637,59.410026
3,Raven.Server,14772,19.230118
1,Raven.Client,6019,7.835505
7,Voron,3694,4.808831
5,Sparrow,2712,3.530469
6,Sparrow.Server,2433,3.167268
0,Corax,1505,1.959202
2,Raven.Embedded,45,0.058581


In [75]:
f_commits.get_authors_of_project('Voron')

,author,changes,perc
1,Federico Lois,2698,73.037358
0,Arkadiusz Paliński,878,23.768273
6,egor,67,1.813752
4,Maciej Aszyk,21,0.568489
5,Paweł Pekról,16,0.433135
2,Grisha Kotler,13,0.351922
3,Karmel Indych,1,0.027071


In [76]:
f_commits.get_projects_of_author('Federico Lois')

,code_project,changes,perc
4,Voron,2698,45.884354
2,Sparrow,1131,19.234694
1,Raven.Server,1030,17.517007
3,Sparrow.Server,946,16.088435
0,Corax,75,1.275510


In [92]:
from pyvis.network import Network
import networkx as nx

class RepoGraph:
    def __init__(self):
        self.G = nx.Graph()
        
    def add_author(self, name):
        self.G.add_node(name, label=name, group='author', title=name)
        
    def has_author(self, name):
        return self.G.has_node(name)
    
    def add_project(self, name):
        self.G.add_node(name, label=name, group='project', title=name)
        
    def add_edge(self, author, project, weight):
        self.G.add_edge(author, project, weight=weight)
    
    def render(self):
        net = Network(notebook=True, height="500px", width="500px")
        net.from_nx(self.G)
        
        net.toggle_physics(True)
        
        for node in net.nodes:
            node_group = node.get('group', 'Unknown')
            if node_group == 'author':
                node['color'] = "#97C2FC"
                node['shape'] = 'dot'
            elif node_group == 'project':
                node['color'] = "#FFC0CB"
                node['shape'] = 'triangle'
        
        net.set_options("""
            var options = {
              "interaction": {
                "zoomView": false
              }
            }
            """)
        
        return net
        #net.show("graph.html")
        

rg = RepoGraph()

In [93]:
import math

def add_project(project_name, commits, rg):
    rg.add_project(project_name)
    authors = commits.get_authors_of_project(project_name)
    
    for index, row in authors.iterrows():
        author = row['author']
        changes = row['changes']  
        rg.add_author(author)
        rg.add_edge(author, project_name, weight=math.log1p(changes))
        
def add_author(author_name, commits, rg):
    rg.add_author(author_name)
    projects = commits.get_projects_of_author(author_name)
    
    for index, row in projects.iterrows():
        project = row['code_project']
        other_authors = commits.get_authors_of_project(project)
        for o_index, o_row in other_authors.iterrows():
            other_author = o_row['author']
            other_changes = o_row['changes']
            if rg.has_author(other_author):
                rg.add_edge(other_author, project, weight=math.log1p(other_changes))
        
        changes = row['changes']  
        rg.add_project(project)
        rg.add_edge(author_name, project, weight=math.log1p(changes))
    

In [94]:
add_project('Voron', commits, rg)
rg.render().show('graph.html')

graph.html


In [95]:
add_project('Corax', commits, rg)
rg.render().show('graph.html')

graph.html


In [96]:
add_author('Maciej Aszyk', commits, rg)
rg.render().show('graph.html')

graph.html


In [ ]:
add_project('Raven.Server', commits, rg)
rg.render().show('graph.html')